## 1. Setup & Installation

In [ ]:
print(" Installing required packages...\n")

!pip install -q transformers datasets torch accelerate huggingface_hub tqdm scikit-learn matplotlib seaborn pandas

print(" Installation complete!\n")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
import json
import random
from tqdm.auto import tqdm
from datetime import datetime
from typing import Dict, List, Tuple, Optional
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from huggingface_hub import HfApi, create_repo, login
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


warnings.filterwarnings('ignore')


SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("="*80)
print(" BAD Classifier Training (IMPROVED & BALANCED)")
print("="*80)
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
print("="*80 + "\n")

## 2. Improved Configuration

In [ ]:
class TrainingConfig:


    # Model
    base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    max_length = 512

    # Data
    bbq_dataset_name = "bitlabsdb/BBQ_dataset"
    bbq_target_loc_dataset = "bitlabsdb/BBQ_Target_Loc_Dataset"
    num_bbq_samples = 58942
    train_val_split = 0.8

    # Sampling
    use_balanced_sampling = True
    target_samples_per_class = 25000
    min_samples_per_class = 5000

    SEED = 42
    # Training Hyperparameters
    batch_size = 128
    num_epochs = 100
    learning_rate = 5e-4
    weight_decay = 1e-4
    early_stopping_patience = 15
    gradient_clip_norm = 1.0


    # TinyLlama has 22 layers. We scan a wider range to find the geometric sweet spot.
    candidate_layers_range = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


    dropout_rate = 0.05

    # Labels
    LABEL_BIASED = 0
    LABEL_UNBIASED = 1

    # HuggingFace
    hf_repo_name = "bitlabsdb/bad-classifier-tinyllama-fairsteer-v2"
    hf_private = False

    # Local
    local_save_dir = "./bad_model_fairsteer_v2"

config = TrainingConfig()

print("  FairSteer-Aligned Training Configuration (Updated):")
print("="*80)
print(f"Base Model:         {config.base_model_name}")
print(f"Splitting Strategy: Group-wise (prevents data leakage)")
print(f"Feature Proc:       Standardization (Zero-mean, Unit-variance)")
print(f"Expanded Layers:    {config.candidate_layers_range}")
print("="*80 + "\n")

## 3. Model Definition

In [ ]:
class BADClassifier(nn.Module):
    """
    Biased Activation Detection (BAD) Classifier - FairSteer Aligned

    Architecture: Dropout -> Linear layer -> Sigmoid

    Changes:
    - Added Dropout (p=0.05 default) to prevent overfitting and 'lazy' feature selection.
    """

    def __init__(self, input_dim: int, dropout_rate: float = 0.05):
        super().__init__()

        # dropout layer
        self.dropout = nn.Dropout(p=dropout_rate)

        # 2. Linear Layer
        self.linear = nn.Linear(input_dim, 1)
        self.input_dim = input_dim

        # Xavier initialization for stable training
        nn.init.xavier_uniform_(self.linear.weight)
        nn.init.zeros_(self.linear.bias)

    def forward(self, x):
        """
        Forward pass: Dropout -> Linear -> Logits
        """
        # Apply dropout first
        x = self.dropout(x)
        return self.linear(x)

    def predict_proba(self, x):
        """
        Get probability of being UNBIASED
        Returns: p(y=1) where y=1 means UNBIASED
        """
        logits = self.forward(x)
        probs = torch.sigmoid(logits).squeeze(-1)
        return probs

    def detect_bias(self, x, threshold: float = 0.5):
        """
        Detect biased activations using FairSteer threshold
        """
        unbiased_prob = self.predict_proba(x)
        is_biased = unbiased_prob < threshold
        return is_biased, unbiased_prob

print("✅ BAD Classifier defined (Architecture: Dropout -> Linear + Sigmoid)\n")

# Load Target Loc dataset to view columns and counting records

In [ ]:

try:
    from datasets import load_dataset
except ImportError:
    !pip install -q datasets
    from datasets import load_dataset

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ---------------------------------------------------------
# 1. DATA LOADING
# ---------------------------------------------------------
print("="*80)
print(" 📥 LOADING DATASET & PREPARING VISUALIZATIONS")
print("="*80 + "\n")

try:
    # Try normal loading first
    print("Attempting standard load...")
    dataset = load_dataset("bitlabsdb/BBQ_Target_Loc_Dataset")
    df = dataset['train'].to_pandas()

except Exception as e:
    # If error (the 'missing' value issue), use streaming method
    print("⚠️ Standard load failed (likely type error). Using streaming method...")
    dataset = load_dataset("bitlabsdb/BBQ_Target_Loc_Dataset", split='train', streaming=True)

    # Convert streaming dataset to pandas
    data_list = []
    for example in dataset:
        data_list.append(example)

    df = pd.DataFrame(data_list)

# ---------------------------------------------------------
# 2. CLEANING
# ---------------------------------------------------------
print("Cleaning data...")
# Fix 'missing' string values in numeric columns
for col in ['label', 'target_loc', 'target_loc_word']:
    if col in df.columns:
        df[col] = df[col].replace('missing', pd.NA)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Filter out records where target_loc is N/A or empty
if 'target_loc' in df.columns:
    original_count = len(df)
    df = df[df['target_loc'].notna()]
    # Ensure target_loc is integer for cleaner plotting
    df['target_loc'] = df['target_loc'].astype(int)
    filtered_count = original_count - len(df)
    print(f"   🔧 Filtered out {filtered_count:,} records with missing target_loc")
else:
    print(f"   ⚠️ Column 'target_loc' not found in dataset")

print(f"✅ Final dataset ready: {len(df):,} rows × {len(df.columns)} columns")

# ---------------------------------------------------------
# 3. VISUALIZATION
# ---------------------------------------------------------
print("\n📊 Generating Distribution Plots...")
sns.set_theme(style="whitegrid")

# Determine layout based on available columns
# We try to find 'category' and 'context_condition' if they exist
has_category = 'category' in df.columns
has_context = 'context_condition' in df.columns

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
plt.subplots_adjust(hspace=0.4, wspace=0.3)

# --- PLOT 1: Target Location Distribution (The most important check) ---
if 'target_loc' in df.columns:
    sns.countplot(x='target_loc', data=df, ax=axes[0, 0], palette="viridis")
    axes[0, 0].set_title('Distribution of Target Locations (0=A, 1=B, 2=C)', fontsize=13)
    axes[0, 0].set_xlabel('Target Location')
    axes[0, 0].set_ylabel('Count')
    axes[0, 0].bar_label(axes[0, 0].containers[0])

# --- PLOT 2: Category Distribution ---
if has_category:
    # Get top categories to prevent overcrowding if there are too many
    top_cats = df['category'].value_counts().head(15)
    sns.barplot(x=top_cats.values, y=top_cats.index, ax=axes[0, 1], palette="magma")
    axes[0, 1].set_title('Distribution by Category', fontsize=13)
    axes[0, 1].set_xlabel('Count')
else:
    axes[0, 1].text(0.5, 0.5, 'Category column missing', ha='center', fontsize=12)

# --- PLOT 3: Context Condition ---
if has_context:
    sns.countplot(x='context_condition', data=df, ax=axes[1, 0], palette="coolwarm")
    axes[1, 0].set_title('Context: Ambiguous vs Disambiguated', fontsize=13)
    axes[1, 0].bar_label(axes[1, 0].containers[0])
else:
    axes[1, 0].text(0.5, 0.5, 'Context column missing', ha='center', fontsize=12)

# --- PLOT 4: Heatmap (Category vs Target) or Label Distribution ---
if has_category and 'target_loc' in df.columns:
    # Check for bias: Do certain categories prefer certain answers?
    cross_tab = pd.crosstab(df['category'], df['target_loc'])
    sns.heatmap(cross_tab, annot=True, fmt='d', cmap="YlGnBu", ax=axes[1, 1])
    axes[1, 1].set_title('Heatmap: Category vs Target Location', fontsize=13)
    axes[1, 1].set_ylabel('')
elif 'label' in df.columns:
    # Fallback: Plot the 'label' column if category missing
    sns.histplot(df['label'].dropna(), ax=axes[1, 1], bins=10, kde=True)
    axes[1, 1].set_title('Label Distribution', fontsize=13)
else:
    axes[1, 1].text(0.5, 0.5, 'Not enough data for correlation', ha='center', fontsize=12)

plt.show()

# ---------------------------------------------------------
# 4. PREVIEW
# ---------------------------------------------------------
print("="*80)
print("TOP 20 ROWS")
print("="*80 + "\n")
display(df.head(5))

print(f"\n💡 DataFrame 'df' ready with {len(df):,} records!")

## 4. Data Loading

In [ ]:
def load_and_merge_bbq_with_targetloc(config: TrainingConfig) -> pd.DataFrame:
    """Load BBQ dataset and merge with target_loc"""

    print("="*80)
    print(" Loading BBQ Dataset with Target_Loc")
    print("="*80 + "\n")

    # Load BBQ dataset
    print("Loading BBQ dataset...")
    try:
        bbq_dataset = load_dataset(config.bbq_dataset_name)
        print(f" Loaded BBQ: {config.bbq_dataset_name}")
    except:
        bbq_dataset = load_dataset("nyu-mll/BBQ")
        print(f" Loaded BBQ: nyu-mll/BBQ")

    # Load Target_Loc
    print("\nLoading Target_Loc dataset...")
    from huggingface_hub import hf_hub_download


    csv_path = hf_hub_download(
        repo_id=config.bbq_target_loc_dataset,
        filename="Untitled spreadsheet - additional_metadata.csv",
        repo_type="dataset"
    )

    targetloc_df = pd.read_csv(csv_path, keep_default_na=False)
    targetloc_df['target_loc'] = pd.to_numeric(targetloc_df['target_loc'], errors='coerce')

    # Deduplicate
    if targetloc_df['example_id'].duplicated().any():
        targetloc_df = targetloc_df.drop_duplicates(subset=['example_id'], keep='first')

    print(f" Loaded Target_Loc")

    # Merge
    bbq_df = pd.DataFrame(bbq_dataset['train'])
    merged_df = pd.merge(
        bbq_df,
        targetloc_df[['example_id', 'target_loc']],
        on='example_id',
        how='inner'
    )

    # Clean
    merged_df = merged_df[merged_df['target_loc'].notnull()]
    merged_df = merged_df[merged_df['target_loc'].isin([0, 1, 2])]

    if len(merged_df) > config.num_bbq_samples:
        merged_df = merged_df.sample(n=config.num_bbq_samples, random_state=SEED)

    print(f"\n Final BBQ dataset: {len(merged_df):,} rows")
    print("="*80 + "\n")

    return merged_df

# Load BBQ
bbq_merged_df = load_and_merge_bbq_with_targetloc(config)
display(bbq_merged_df.head(20))
print(f"Sample: {len(bbq_merged_df)} BBQ examples loaded\n")



# CREATE GROUP IDENTIFIERS
## Grouping BBQ databse with category and Question Index
## Hashing the category and Question Index


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("\n" + "="*80)
print("🔧 CREATING GROUP IDENTIFIERS & VISUALIZING DISTRIBUTION")
print("="*80 + "\n")

def create_group_identifier(row):
    """
    Create unique group ID to keep paired/similar prompts together.
    Returns both a readable string (for plots) and a hashed ID (for processing).
    """

    group_str = f"{row['category']}-{row['question_index']}"

    return group_str, hash(group_str) % 100000


df[['group_name', 'group_id']] = df.apply(
    lambda row: pd.Series(create_group_identifier(row)), axis=1
)


print(f" Created group identifiers")
print(f"   Total samples: {len(df):,}")
print(f"   Unique groups: {df['group_id'].nunique():,}")
print(f"   Avg samples per group: {len(df) / df['group_id'].nunique():.2f}")

group_sizes = df.groupby('group_name').size()
print(f"   Group size range: {group_sizes.min()} - {group_sizes.max()}")
print(f"   Groups with >1 sample: {(group_sizes > 1).sum():,}")


# Set plot style
sns.set_theme(style="whitegrid")

# Create a figure with 3 subplots
fig, axes = plt.subplots(3, 1, figsize=(12, 18))
plt.subplots_adjust(hspace=0.4)

# --- PLOT 1: Distribution of Group Sizes (Histogram) ---
# This shows if your groups are balanced (e.g., do most groups have 2 questions?)
size_counts = group_sizes.value_counts().sort_index()
sns.barplot(x=size_counts.index, y=size_counts.values, ax=axes[0], palette="viridis")
axes[0].set_title('Distribution of Group Sizes (How many items per group?)', fontsize=14)
axes[0].set_xlabel('Number of Items in Group')
axes[0].set_ylabel('Count of Groups')
axes[0].bar_label(axes[0].containers[0])

# --- PLOT 2: Top 20 Largest Groups by Name ---
# This identifies specific groups that might be "too big" (Outliers)
top_groups = group_sizes.sort_values(ascending=False).head(20)
sns.barplot(x=top_groups.values, y=top_groups.index, ax=axes[1], palette="magma")
axes[1].set_title('Top 20 Largest Groups (Outliers)', fontsize=14)
axes[1].set_xlabel('Number of Samples')
axes[1].set_ylabel('Group Name')
axes[1].bar_label(axes[1].containers[0])

# --- PLOT 3: Number of Unique Groups per Category ---
# This checks if one category dominates the dataset
cat_counts = df.groupby('category')['group_id'].nunique().sort_values(ascending=False)
sns.barplot(x=cat_counts.values, y=cat_counts.index, ax=axes[2], palette="coolwarm")
axes[2].set_title('Number of Unique Groups per Category', fontsize=14)
axes[2].set_xlabel('Unique Group Count')
axes[2].bar_label(axes[2].containers[0])

print("\n📊 Generating plots...")
plt.show()

## 5. Load the LLM Base Model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

print("="*80)
print(" 🧠 LOADING BASE MODEL & INSPECTING HIDDEN STREAMS")
print("="*80 + "\n")

# 1. Load Tokenizer & Model
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map='auto'
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model.eval()

# ------------------------------------------------------------------
# 2. EXTRACT ARCHITECTURE DETAILS
# ------------------------------------------------------------------
# Different models store layer counts in different config attributes
model_config = base_model.config
num_layers = getattr(model_config, "num_hidden_layers", None)
if num_layers is None:
    num_layers = getattr(model_config, "n_layer", None) # GPT style

hidden_size = getattr(model_config, "hidden_size", None)
if hidden_size is None:
    hidden_size = getattr(model_config, "n_embd", "Unknown")

num_heads = getattr(model_config, "num_attention_heads", getattr(model_config, "n_head", "Unknown"))

# ------------------------------------------------------------------
# 3. VISUALIZE THE HIDDEN STATE STREAM
# ------------------------------------------------------------------
print(f"✅ Loaded Model: {config.base_model_name}")
print(f"   • Model Type: {model_config.model_type}")
print(f"   • Vocabulary: {model_config.vocab_size:,} tokens")
print("-" * 40)
print(f"🌊 HIDDEN STATE STREAM STRUCTURE")
print("-" * 40)

# Visualizing the Data Flow
print(f"   [INPUT] (Batch, Seq_Len)")
print(f"      ⬇")
print(f"   [EMBEDDING] ➜ Transforms tokens to vector size {hidden_size}")
print(f"      ⬇")

# The "Stream" - The stack of Transformer Blocks
if num_layers:
    # Draw the stack
    print(f"   ╔════════════════════════════════════╗")
    print(f"   ║  HIDDEN LAYERS (The Stream)        ║")
    print(f"   ╠════════════════════════════════════╣")
    print(f"   ║  Layer  1 / {num_layers:<3} (Start)           ║")
    print(f"   ║  Layer  2 / {num_layers:<3}                   ║")
    print(f"   ║    ... (Identical Blocks) ...      ║")
    print(f"   ║    ... (Attn + MLP + Norm)...      ║")
    print(f"   ║  Layer {num_layers} / {num_layers:<3} (End)             ║")
    print(f"   ╚════════════════════════════════════╝")
    print(f"      ⬇ (Stream Size: {hidden_size} dims, {num_heads} Attn Heads)")
else:
    print("   [Unknown Layer Structure - could not detect num_hidden_layers]")

print(f"   [OUTPUT HEAD] ➜ Projects {hidden_size} back to {model_config.vocab_size}")
print(f"      ⬇")
print(f"   [LOGITS] (Probabilities for next token)")

print("\n" + "="*80)



## 6. Extract BBQ Activations

In [ ]:
def extract_bbq_activations(
    model,
    tokenizer,
    merged_df: pd.DataFrame,
    config: TrainingConfig,
    layer_idx: int
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Extract activations with STRICT labeling logic to ensure clean steering vectors.

    Logic Update:
    - Biased (0): Model selects the Stereotypical Target (target_loc).
    - Unbiased (1): Model selects the Neutral/Unknown option (Dynamically found).
    - Skip: Model selects the Counter-Stereotypical group or we can't find neutral option.
    """
    model.eval()
    activations = []
    labels = []
    group_ids = []

    stats = {'biased': 0, 'unbiased': 0, 'skipped': 0, 'failed': 0}

    # Ensure we have a group_id column
    if 'group_id' not in merged_df.columns:
        print("Generating group IDs...")
        merged_df['group_id'] = merged_df.apply(
            lambda x: hash(f"{x['category']}-{x['question_index']}") % 100000, axis=1
        )

    # Pre-tokenization map for A, B, C
    token_map = {
        0: tokenizer.encode('A', add_special_tokens=False)[0],
        1: tokenizer.encode('B', add_special_tokens=False)[0],
        2: tokenizer.encode('C', add_special_tokens=False)[0]
    }

    print(f"Extraction Layer {layer_idx} | Strict Mode: ON (Robust Neutral Detection)")

    # EXPANDED list of neutral keywords to catch all valid unbiased answers
    neutral_keywords = [
        "unknown",
        "can't be determined",
        "cannot be determined",
        "undetermined",
        "not enough info",
        "not enough information",
        "not answerable",
        "not specified",
        "not stated",
        "insufficient information",
        "neither",
        "none of the above"
    ]

    for idx, row in tqdm(merged_df.iterrows(), total=len(merged_df), desc=f"Layer {layer_idx}"):
        try:
            context = row.get('context', '')
            question = row.get('question', '')
            ans0 = str(row.get('ans0', ''))
            ans1 = str(row.get('ans1', ''))
            ans2 = str(row.get('ans2', ''))
            target_loc = int(row.get('target_loc', -1))
            context_condition = row.get('context_condition', '')

            #capture the failed records by filtering out target_loc and context_condition are empty or -1.
            if target_loc == -1 or context_condition == '':
                stats['failed'] += 1
                continue

            # ----------------------------------------------------------------
            # 1. Identify the Neutral / Unknown Index dynamically (STRICT)
            # ----------------------------------------------------------------
            choices = [ans0, ans1, ans2]
            unknown_idx = -1

            for i, choice in enumerate(choices):
                c_lower = choice.lower().strip()
                # Check against the expanded list of neutral phrases
                if any(kw in c_lower for kw in neutral_keywords):
                    unknown_idx = i
                    break

            # STRICT RULE: If we cannot explicitly find the "Neutral" text,
            # we DO NOT guess. We skip this sample to prevent label noise.
            if unknown_idx == -1:
                stats['skipped'] += 1
                continue

            # ----------------------------------------------------------------
            # 2. Forward Pass
            # ----------------------------------------------------------------
            prompt = f"{context} {question}\nA. {ans0}\nB. {ans1}\nC. {ans2}\nAnswer:"

            inputs = tokenizer(
                prompt,
                return_tensors="pt",
                max_length=config.max_length,
                truncation=True,
                padding=True
            ).to(device)

            with torch.no_grad():
                outputs = model(**inputs, output_hidden_states=True, return_dict=True)

            # Extract last token hidden state
            hidden_states = outputs.hidden_states[layer_idx]
            last_token_hidden = hidden_states[:, -1, :].cpu().numpy()[0]

            # ----------------------------------------------------------------
            # 3. Prediction & Strict Labeling
            # ----------------------------------------------------------------
            logits = outputs.logits[:, -1, :]
            probs = torch.softmax(logits, dim=-1)
            pred_probs = [probs[0, token_map[i]].item() for i in range(3)]
            predicted_idx = np.argmax(pred_probs)

            label = -1

            if context_condition == 'ambig':
                if predicted_idx == target_loc:
                    # Model picked the stereotype -> BIASED
                    label = config.LABEL_BIASED
                    stats['biased'] += 1
                elif predicted_idx == unknown_idx:
                    # Model picked the neutral option -> UNBIASED
                    label = config.LABEL_UNBIASED
                    stats['unbiased'] += 1
                else:
                    # Model picked the counter-stereotypical group.
                    stats['skipped'] += 1

            elif context_condition == 'disambig':
                # Use ground truth logic for disambiguated contexts
                correct_label = int(row.get('label', -1))

                if correct_label != -1:
                    if predicted_idx == target_loc and predicted_idx != correct_label:
                        # Stereotype is wrong, but model picked it -> BIASED
                        label = config.LABEL_BIASED
                        stats['biased'] += 1
                    elif predicted_idx == correct_label:
                        # Model picked correct answer -> UNBIASED
                        label = config.LABEL_UNBIASED
                        stats['unbiased'] += 1
                    else:
                        stats['skipped'] += 1
                else:
                    stats['skipped'] += 1

            # 4. Save to buffer
            if label != -1:
                activations.append(last_token_hidden)
                labels.append(label)
                group_ids.append(row['group_id'])

        except Exception as e:
            stats['failed'] += 1
            continue

    print(f"   Stats: Biased={stats['biased']}, Unbiased={stats['unbiased']}, Skipped={stats['skipped']}, Failed/Err={stats['failed']}")

    return np.array(activations), np.array(labels), np.array(group_ids)

print("✅ FairSteer-aligned extraction function (Strict Dynamic Neutral Detection) defined\n")

## 7. Balance Dataset for training uses

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupShuffleSplit
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns

def visualize_pipeline_stats(X_raw, X_scaled, y_train, y_val, groups_train, groups_val):
    """Helper to verify data health"""
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    # 1. Class Balance
    sns.countplot(x=y_train, ax=axes[0], color='skyblue', label='Train')
    axes[0].set_title(f"Class Balance (Train: {len(y_train)})")
    axes[0].set_xticklabels(['Biased', 'Unbiased'])

    # 2. Distribution Shift (First dimension as proxy)
    # We use X_raw vs X_scaled. They MUST be the same length.
    if len(X_raw) > 0 and len(X_scaled) > 0:
        # Pick a random sample index valid for both
        idx = np.random.randint(0, len(X_raw))

        # Compare distribution of features for one sample
        sns.kdeplot(X_raw[idx].flatten(), ax=axes[1], color='red', label='Raw (Bal)', fill=True, alpha=0.3)
        sns.kdeplot(X_scaled[idx].flatten(), ax=axes[1], color='blue', label='Scaled', fill=True, alpha=0.3)
        axes[1].set_title("Feature Distribution (Single Sample)")
        axes[1].legend()

    plt.tight_layout()
    plt.show()

def prepare_data_pipeline(activations, labels, group_ids, config):
    """
    Implements Dr. Raj's Pipeline:
    1. Split by Group (prevent leakage)
    2. Balance (downsample majority)
    3. Standardize (Zero-mean, Unit-variance)
    """
    print("\n" + "="*80)
    print("  🔧 DATA PIPELINE EXECUTION")
    print("="*80)

    # ---------------------------------------------------------
    # 1. Group-wise Split (Prevent Leakage)
    # ---------------------------------------------------------
    gss = GroupShuffleSplit(n_splits=1, train_size=config.train_val_split, random_state=config.SEED)
    train_idx, val_idx = next(gss.split(activations, labels, groups=group_ids))

    X_train_raw, y_train_raw, g_train_raw = activations[train_idx], labels[train_idx], group_ids[train_idx]
    X_val_raw, y_val_raw, g_val_raw = activations[val_idx], labels[val_idx], group_ids[val_idx]

    # Overlap check
    train_groups = set(g_train_raw)
    val_groups = set(g_val_raw)
    overlap = train_groups.intersection(val_groups)

    print(f"  Step 1: Split (Group-wise)")
    print(f"     Train Samples: {len(X_train_raw):,}")
    print(f"     Val Samples:   {len(X_val_raw):,}")
    if len(overlap) == 0:
        print(f"     ✅ LEAKAGE CHECK PASSED: 0 overlapping groups.")
    else:
        print(f"     ❌ WARNING: {len(overlap)} GROUPS LEAKED!")

    # ---------------------------------------------------------
    # 2. Balance Data (Downsample majority class)
    # ---------------------------------------------------------
    def balance_subset(X, y, g):
        X_biased = X[y == config.LABEL_BIASED]
        X_unbiased = X[y == config.LABEL_UNBIASED]
        g_biased = g[y == config.LABEL_BIASED]
        g_unbiased = g[y == config.LABEL_UNBIASED]

        n_samples = min(len(X_biased), len(X_unbiased))
        if config.target_samples_per_class:
            n_samples = min(n_samples, config.target_samples_per_class)

        # Resample
        X_b_res, g_b_res = resample(X_biased, g_biased, n_samples=n_samples, random_state=config.SEED, replace=False)
        X_u_res, g_u_res = resample(X_unbiased, g_unbiased, n_samples=n_samples, random_state=config.SEED, replace=False)

        X_bal = np.vstack([X_b_res, X_u_res])
        y_bal = np.array([config.LABEL_BIASED]*n_samples + [config.LABEL_UNBIASED]*n_samples)
        g_bal = np.concatenate([g_b_res, g_u_res])

        # Shuffle
        p = np.random.permutation(len(X_bal))
        return X_bal[p], y_bal[p], g_bal[p]

    X_train_bal, y_train_bal, g_train_bal = balance_subset(X_train_raw, y_train_raw, g_train_raw)
    X_val_bal, y_val_bal, g_val_bal = balance_subset(X_val_raw, y_val_raw, g_val_raw)

    print(f"  Step 2: Balanced")
    print(f"     Train: {len(X_train_bal):,} (Groups: {len(np.unique(g_train_bal)):,})")
    print(f"     Val:   {len(X_val_bal):,} (Groups: {len(np.unique(g_val_bal)):,})")

    # ---------------------------------------------------------
    # 3. Standardization (Critical for Linear Probe)
    # ---------------------------------------------------------
    # Convert to float32 to avoid 'inf' variance in float16
    X_train_bal = X_train_bal.astype(np.float32)
    X_val_bal = X_val_bal.astype(np.float32)

    # Clean any existing NaNs/Infs
    if not np.isfinite(X_train_bal).all():
        print("     ⚠️ Warning: Found NaNs/Infs in activations. Replacing with 0.")
        X_train_bal = np.nan_to_num(X_train_bal)
        X_val_bal = np.nan_to_num(X_val_bal)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_bal)
    X_val_scaled = scaler.transform(X_val_bal)

    print(f"  Step 3: Standardized")
    print(f"     Before Scaling -> Mean: {np.mean(X_train_bal):.4f}, Std: {np.std(X_train_bal):.4f}")
    print(f"     After Scaling  -> Mean: {np.mean(X_train_scaled):.4f},  Std: {np.std(X_train_scaled):.4f}")

    # ---------------------------------------------------------
    # 4. Visualize (Pass the BALANCED raw data)
    # ---------------------------------------------------------
    # We pass X_train_bal (not X_train_raw) so sizes match X_train_scaled
    visualize_pipeline_stats(X_train_bal, X_train_scaled, y_train_bal, y_val_bal, g_train_bal, g_val_bal)

    return X_train_scaled, y_train_bal, X_val_scaled, y_val_bal, scaler

print("✅ Data Pipeline defined (Split -> Balance -> Standardize) with Visualization Fix\n")

## 8. Extract and Balance for All Layers

In [ ]:
import os
import pickle
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm.auto import tqdm

print("="*80)
print(" Extracting & Processing Data (Optimized Single-Pass with Visuals)")
print("="*80 + "\n")


# Get absolute paths to avoid confusion
cwd = os.getcwd()
abs_save_dir = os.path.abspath(config.local_save_dir)
cache_dir = os.path.join(abs_save_dir, "cache_processed_layers")

print(f"📍 Current Working Dir:  {cwd}")
print(f"📂 Expected Cache Dir:   {cache_dir}")

os.makedirs(cache_dir, exist_ok=True)

def get_cache_path(layer_idx):
    return os.path.join(cache_dir, f"layer_{layer_idx}_data.pkl")


missing_layers = []
found_layers = []

print(f"\n🔍 Checking for {len(config.candidate_layers_range)} layer files...")

for l in config.candidate_layers_range:
    fpath = get_cache_path(l)
    if os.path.exists(fpath):
        # Check if file is not empty (0 bytes)
        if os.path.getsize(fpath) > 0:
            found_layers.append(l)
            # print(f"   ✅ Found Layer {l}") # Optional verbose
        else:
            print(f"   ⚠️ Layer {l} found but is EMPTY (0 bytes). Will re-extract.")
            missing_layers.append(l)
    else:
        # print(f"   ❌ Missing Layer {l}") # Optional verbose
        missing_layers.append(l)

# Decision Logic
all_layer_data = {}

if len(missing_layers) == 0:
    print(f"\n⚡ CACHE COMPLETE! Loading data for {len(found_layers)} layers...")
    for layer_idx in tqdm(config.candidate_layers_range, desc="Loading Cache"):
        with open(get_cache_path(layer_idx), "rb") as f:
            all_layer_data[layer_idx] = pickle.load(f)
    print("✅ All data loaded successfully. Skipping extraction.")

else:
    print(f"\n🐢 Cache incomplete ({len(missing_layers)} missing).")
    print(f"   Missing Layers: {missing_layers}")
    print("   🚀 Starting SINGLE-PASS extraction...")

    # -------------------------------------------------------
    # OPTIMIZED INFERENCE LOOP (Runs only if cache missing)
    # -------------------------------------------------------

    # Buffers
    layer_buffers = {l: [] for l in config.candidate_layers_range}
    labels_buffer = []
    groups_buffer = []
    stats = {'biased': 0, 'unbiased': 0, 'failed': 0}

    # Prepare Data
    extraction_df = bbq_merged_df.copy()
    if 'group_id' not in extraction_df.columns:
         extraction_df['group_id'] = extraction_df.apply(
             lambda x: hash(f"{x['category']}-{x['question_index']}") % 100000, axis=1
         )

    base_model.eval()

    # Live Progress Bar
    pbar = tqdm(extraction_df.iterrows(), total=len(extraction_df), desc="Extracting")

    for idx, row in pbar:
        try:
            # Inputs
            context = row.get('context', '')
            question = row.get('question', '')
            ans0, ans1, ans2 = row.get('ans0', ''), row.get('ans1', ''), row.get('ans2', '')
            target_loc = int(row.get('target_loc', -1))
            context_condition = row.get('context_condition', '')

            if target_loc == -1 or context_condition == '':
                stats['failed'] += 1
                continue

            prompt = f"{context} {question}\nA. {ans0}\nB. {ans1}\nC. {ans2}\nAnswer:"
            inputs = tokenizer(prompt, return_tensors="pt", max_length=config.max_length, truncation=True).to(device)

            # Forward
            with torch.no_grad():
                outputs = base_model(**inputs, output_hidden_states=True, return_dict=True)

            # Labeling
            logits = outputs.logits[:, -1, :]
            probs = torch.softmax(logits, dim=-1)
            t_map = [tokenizer.encode(k, add_special_tokens=False)[0] for k in ['A', 'B', 'C']]
            pred_probs = [probs[0, t].item() for t in t_map]
            predicted_idx = np.argmax(pred_probs)

            label = -1
            if context_condition == 'ambig':
                label = config.LABEL_BIASED if predicted_idx == target_loc else config.LABEL_UNBIASED
            elif context_condition == 'disambig':
                label = config.LABEL_BIASED if predicted_idx == 2 else config.LABEL_UNBIASED

            # Store
            if label != -1:
                labels_buffer.append(label)
                groups_buffer.append(row['group_id'])

                if label == config.LABEL_BIASED: stats['biased'] += 1
                else: stats['unbiased'] += 1

                # Extract from ALL layers
                for layer_idx in config.candidate_layers_range:
                    hidden = outputs.hidden_states[layer_idx][:, -1, :].cpu().numpy()[0]
                    layer_buffers[layer_idx].append(hidden)

            if (stats['biased'] + stats['unbiased']) % 100 == 0:
                pbar.set_postfix({'B': stats['biased'], 'U': stats['unbiased']})

        except Exception as e:
            stats['failed'] += 1
            continue

    # -------------------------------------------------------
    # PROCESSING & SAVING
    # -------------------------------------------------------
    print("\n" + "="*80)
    print(" 💾 PROCESSING & SAVING TO DISK")
    print("="*80)

    final_labels = np.array(labels_buffer)
    final_groups = np.array(groups_buffer)

    for layer_idx in config.candidate_layers_range:
        print(f"  > Processing Layer {layer_idx}...")

        final_acts = np.array(layer_buffers[layer_idx])

        # Run Dr. Raj's Pipeline
        X_train, y_train, X_val, y_val, scaler = prepare_data_pipeline(
            final_acts, final_labels, final_groups, config
        )

        layer_data = {
            'X_train': X_train, 'y_train': y_train,
            'X_val': X_val, 'y_val': y_val,
            'scaler': scaler
        }

        # Save
        save_path = get_cache_path(layer_idx)
        with open(save_path, "wb") as f:
            pickle.dump(layer_data, f)

        all_layer_data[layer_idx] = layer_data
        print(f"    ✅ Saved: {save_path}")

print(f"\n✅ ALL SYSTEMS GO. Data Ready.")

# 9. Training Function
## Start trainig the BAD MODEL with the ready balanced dataset

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import (
    balanced_accuracy_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    auc
)
from typing import Tuple, Dict

# ============================================================================
# 1. VISUALIZATION HELPER FUNCTIONS
# ============================================================================
def plot_training_history(history):
    """Plots the evolution of Loss, AUC, and Accuracy over epochs."""
    epochs = range(1, len(history['train_loss']) + 1)

    sns.set_theme(style="whitegrid")
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))

    # Plot 1: Loss Curve
    sns.lineplot(x=epochs, y=history['train_loss'], ax=axes[0], marker='o', label='Train Loss', color='red')
    axes[0].set_title("Training Loss Evolution")
    axes[0].set_xlabel("Epoch")
    axes[0].set_ylabel("Loss")

    # Plot 2: Metrics Curve
    sns.lineplot(x=epochs, y=history['val_bal_acc'], ax=axes[1], marker='o', label='Val Balanced Acc', color='blue')
    sns.lineplot(x=epochs, y=history['val_auc'], ax=axes[1], marker='s', label='Val AUC', color='green')
    axes[1].set_title("Validation Metrics Evolution")
    axes[1].set_xlabel("Epoch")
    axes[1].set_ylabel("Score")
    axes[1].set_ylim(0.4, 1.05)
    axes[1].legend()

    plt.tight_layout()
    plt.show()

def plot_final_evaluation(y_true, y_probs, y_preds):
    """Plots Confusion Matrix and ROC Curve for the best model."""
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Plot 1: Confusion Matrix
    cm = confusion_matrix(y_true, y_preds)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0], cbar=False,
                xticklabels=['Unbiased (0)', 'Biased (1)'],
                yticklabels=['Unbiased (0)', 'Biased (1)'])
    axes[0].set_title("Confusion Matrix (Best Model)")
    axes[0].set_xlabel("Predicted")
    axes[0].set_ylabel("Actual")

    # Plot 2: ROC Curve
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    axes[1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    axes[1].set_xlim([0.0, 1.0])
    axes[1].set_ylim([0.0, 1.05])
    axes[1].set_xlabel('False Positive Rate')
    axes[1].set_ylabel('True Positive Rate')
    axes[1].set_title('Receiver Operating Characteristic (ROC)')
    axes[1].legend(loc="lower right")

    plt.tight_layout()
    plt.show()


def train_bad_classifier(
    X_train, y_train, X_val, y_val,
    config,
    layer_idx: int
):
    """
    Trains a probe/classifier with comprehensive logging and visualization.
    """
    print("="*80)
    print(f" 🚀 TRAINING BAD CLASSIFIER - LAYER {layer_idx}")
    print("="*80)

    # 1. Setup Data
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    X_train_t = torch.tensor(X_train, dtype=torch.float32)
    y_train_t = torch.tensor(y_train, dtype=torch.float32)
    X_val_t = torch.tensor(X_val, dtype=torch.float32)
    y_val_t = torch.tensor(y_val, dtype=torch.float32)

    train_dataset = TensorDataset(X_train_t, y_train_t)
    val_dataset = TensorDataset(X_val_t, y_val_t)

    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)


    model = BADClassifier(
    input_dim=X_train.shape[1],
    dropout_rate=config.dropout_rate).to(device)


    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)


    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=3
    )

    # 3. Training Loop
    best_metric = 0.0
    best_model_state = None
    patience_counter = 0
    history = {'val_auc': [], 'val_bal_acc': [], 'train_loss': []}

    print(f"   Starting training on {device} for {config.num_epochs} epochs...")
    print(f"   {'Epoch':<6} | {'Loss':<8} | {'Bal Acc':<8} | {'AUC':<8} | {'Status'}")
    print("-" * 55)

    for epoch in range(config.num_epochs):
        # --- TRAIN ---
        model.train()
        train_loss = 0

        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            logits = model(batch_X).squeeze(-1)
            loss = criterion(logits, batch_y)
            loss.backward()

            if config.gradient_clip_norm > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.gradient_clip_norm)

            optimizer.step()
            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # --- VALIDATE ---
        model.eval()
        val_probs = []
        val_targets = []

        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X = batch_X.to(device)
                logits = model(batch_X).squeeze(-1)
                probs = torch.sigmoid(logits)
                val_probs.extend(probs.cpu().numpy())
                val_targets.extend(batch_y.cpu().numpy())

        # --- METRICS ---
        val_preds = [1 if p >= 0.5 else 0 for p in val_probs]
        val_bal_acc = balanced_accuracy_score(val_targets, val_preds)

        try:
            val_auc = roc_auc_score(val_targets, val_probs)
        except ValueError:
            val_auc = 0.5

        # Store history
        history['val_bal_acc'].append(val_bal_acc)
        history['val_auc'].append(val_auc)
        history['train_loss'].append(avg_train_loss)

        # Save Best
        save_msg = ""
        if val_bal_acc > best_metric:
            best_metric = val_bal_acc
            best_model_state = model.state_dict().copy()
            patience_counter = 0
            save_msg = "⭐ New Best"
        else:
            patience_counter += 1

        # Print Status
        print(f"   {epoch+1:<6} | {avg_train_loss:.4f}   | {val_bal_acc:.4f}   | {val_auc:.4f}   | {save_msg}")

        # Scheduler & Early Stop
        scheduler.step(val_bal_acc)

        if patience_counter >= config.early_stopping_patience:
            print(f"\n🛑 Early stopping triggered at epoch {epoch+1}")
            break

    # 4. Final Evaluation & Visualization
    print("\n" + "="*80)
    print(" 📊 EVALUATING BEST MODEL")
    print("="*80)

    model.load_state_dict(best_model_state)

    # Final predictions for reporting
    model.eval()
    final_probs = []
    final_targets = []
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X = batch_X.to(device)
            logits = model(batch_X).squeeze(-1)
            probs = torch.sigmoid(logits)
            final_probs.extend(probs.cpu().numpy())
            final_targets.extend(batch_y.cpu().numpy())

    final_preds = [1 if p >= 0.5 else 0 for p in final_probs]

    print("\nClassification Report:")
    print(classification_report(final_targets, final_preds, target_names=['Biased', 'Unbiased']))

    print("Generating Plots...")
    plot_training_history(history)
    plot_final_evaluation(final_targets, final_probs, final_preds)

    return model, best_metric, history

print("✅ Training function updated (Fixed PyTorch verbose error).")

## 10. Train All Layers & Select Best

In [ ]:
import gc
import torch

print("="*80)
print(" 🧠 OPTIMIZATION: CLEARING VRAM BEFORE TRAINING")
print("="*80 + "\n")


# We have already extracted activations to CPU/Disk. We don't need the LLM on GPU anymore.
if 'base_model' in globals():
    print(" 🗑️  Unloading Base Model to free GPU memory...")
    del base_model
if 'tokenizer' in globals():
    del tokenizer

# 2. Force Garbage Collection
gc.collect()
torch.cuda.empty_cache()

# Verify Memory
if torch.cuda.is_available():
    free_mem = torch.cuda.mem_get_info()[0] / 1024**3
    print(f" 💾 GPU Memory Free: {free_mem:.2f} GB (Should be high now)")

print("\n" + "="*80)
print(" Training BAD Classifiers for All Layers")
print("="*80 + "\n")

layer_results = {}
trained_models = {}
trained_scalers = {}
histories = {}

best_layer = None
best_score = 0.0

# Iterate through candidate layers
for layer_idx in config.candidate_layers_range:
    print(f"\n{'='*60}\n ⚙️  PROCESSING LAYER {layer_idx}\n{'='*60}")

    # Retrieve Data
    if layer_idx not in all_layer_data:
        print(f"❌ Error: Data for Layer {layer_idx} not found. Skipping.")
        continue

    layer_data = all_layer_data[layer_idx]

    # Train
    # Note: The training function moves small batches to GPU, computes gradients,
    # and then releases them. This is very memory efficient.
    try:
        model, score, hist = train_bad_classifier(
            layer_data['X_train'],
            layer_data['y_train'],
            layer_data['X_val'],
            layer_data['y_val'],
            config,
            layer_idx
        )

        # Store Results
        layer_results[layer_idx] = score
        trained_models[layer_idx] = model.cpu() # Move model to CPU to save GPU for next iter
        trained_scalers[layer_idx] = layer_data['scaler']
        histories[layer_idx] = hist

        # Track Best
        if score > best_score:
            best_score = score
            best_layer = layer_idx

    except RuntimeError as e:
        if "out of memory" in str(e):
            print(f"❌ OOM Error on Layer {layer_idx}. Skipping this layer.")
            torch.cuda.empty_cache()
            continue
        else:
            raise e

    # Aggressive Cleanup between layers
    del model
    torch.cuda.empty_cache()

# Final Summary Table
print(f"\n\n{'='*80}")
print(f" 🏆 LAYER SELECTION RESULTS (Metric: Balanced Accuracy)")
print(f"{'='*80}\n")

print(f"{'Layer':<10} {'Bal Acc':<15} {'Improvement':<25} {'Status'}")
print("-" * 70)

for layer in sorted(layer_results.keys()):
    acc = layer_results[layer]
    improvement = (acc - 0.5) / 0.5 * 100

    marker = ""
    if layer == best_layer:
        marker = "⭐⭐ WINNER ⭐⭐"
    elif acc > 0.75:
        marker = "✅ Good Candidate"

    print(f"{layer:<10} {acc:<15.4f} {f'+{improvement:.1f}%':<25} {marker}")

if best_layer is not None:
    print(f"\n{'='*80}")
    print(f" Final Decision: Selected Layer {best_layer}")
    print(f" Best Balanced Accuracy: {best_score:.4f}")
    print(f"{'='*80}\n")

    # Move best model back to GPU for saving/evaluation if needed
    trained_models[best_layer] = trained_models[best_layer].to(device)
else:
    print("\n❌ Training failed for all layers.")

## 11. Visualize Results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

print("="*80)
print(f" 📊 VISUALIZING RESULTS FOR BEST LAYER: {best_layer}")
print("="*80 + "\n")


history = histories[best_layer]
epochs = range(1, len(history['train_loss']) + 1)

# 2. Setup the dashboard
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
plt.suptitle(f"Training Dynamics - Layer {best_layer} (Best Model)", fontsize=16, fontweight='bold')

# Panel 1: Training Optimization (Loss)
# -------------------------------------------------------
loss_data = history['train_loss']
ax1.plot(epochs, loss_data, 'b-', alpha=0.4, label='Raw Batch Loss')

# Add a smoothed trend line if we have enough epochs
if len(loss_data) > 5:
    window = 5
    smoothed = np.convolve(loss_data, np.ones(window)/window, mode='valid')
    ax1.plot(range(window//2 + 1, len(smoothed) + window//2 + 1), smoothed, 'b-', linewidth=2, label='Smoothed Trend')

ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('BCE Loss', fontsize=12)
ax1.set_title('Optimization (Did it converge?)', fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.legend()

# Panel 2: Generalization (Metrics)
# -------------------------------------------------------
# We plot Balanced Acc vs AUC to check for threshold issues
ax2.plot(epochs, history['val_bal_acc'], 'g-', linewidth=2, label='Balanced Acc (Decision)')
ax2.plot(epochs, history['val_auc'], 'purple', linestyle='--', linewidth=2, label='AUC (Separability)')

# Reference lines
ax2.axhline(y=0.5, color='gray', linestyle=':', label='Random Baseline (0.5)')
ax2.axhline(y=best_score, color='red', linestyle='--', alpha=0.5, label=f'Best: {best_score:.4f}')

ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Score (0.0 - 1.0)', fontsize=12)
ax2.set_title('Validation Performance', fontsize=14)
ax2.set_ylim(0.4, 1.0) # Zoom in on relevant range
ax2.legend(loc='lower right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{config.local_save_dir}/training_dashboard.png", dpi=150)
plt.show()

print(f" Saved analysis to: {config.local_save_dir}/training_dashboard.png\n")

## 12. Save Model

In [ ]:
import pickle
import json

print("="*80)
print(" Saving Model & Assets")
print("="*80 + "\n")

os.makedirs(config.local_save_dir, exist_ok=True)

# 1. Save PyTorch Model
torch.save(trained_models[best_layer].state_dict(), f"{config.local_save_dir}/pytorch_model.bin")
print(f" Saved: pytorch_model.bin")

# 2. Save Scaler
# The inference pipeline must load this and apply .transform() before the model
with open(f"{config.local_save_dir}/scaler.pkl", "wb") as f:
    pickle.dump(trained_scalers[best_layer], f)
print(f" Saved: scaler.pkl (StandardScaler)")

# 3. Save Config
model_config = {
    'input_dim': trained_models[best_layer].input_dim,
    'layer_idx': best_layer,
    'base_model': config.base_model_name,
    'best_val_bal_acc': float(best_score),
    'training_method': 'GroupSplit + Standardized + Balanced',
    'training_date': datetime.now().isoformat()
}
with open(f"{config.local_save_dir}/config.json", 'w') as f:
    json.dump(model_config, f, indent=2)
print(f" Saved: config.json")

print(f"\n All assets ready in {config.local_save_dir}")

## 13. Final Summary

In [ ]:
print("="*80)
print(" TRAINING COMPLETE (IMPROVED & BALANCED)")
print("="*80 + "\n")

# Calculate total samples from the best layer's data
best_layer_data = all_layer_data[best_layer]
total_samples = len(best_layer_data['y_train']) + len(best_layer_data['y_val'])

print(" Final Results:")
print(f"   Best Layer:       {best_layer}")
print(f"   Best Bal Acc:     {best_score:.4f}")
print(f"   Baseline:         0.5000 (50% random)")
print(f"   Improvement:      {(best_score - 0.5) / 0.5 * 100:+.1f}% over baseline")
print(f"")
print(f"   Total Samples:    {total_samples:,} (Train + Val)")
print(f"   Class Split:      50% BIASED, 50% UNBIASED (Perfectly Balanced)")
print(f"   Pipeline:         Group Split -> Standardization -> Linear Probe")
print(f"")
print(f"   Save Location:    {config.local_save_dir}")
print(f"")

# Comparison with previous attempt
print(" Comparison with Previous Attempts:")
print(f"   Previous (Raw):   66.88% (Likely overfitting on duplicates)")
print(f"   Current (Bal):    {best_score:.2%} (Robust, Group-split, Standardized)")

# Success Logic
if best_score >= 0.80:
    print(f"\n 🚀 EXCELLENT! Model is highly discriminative. FairSteer will work well.")
elif best_score >= 0.75:
    print(f"\n ✅ SUCCESS! Model is well-trained and ready for deployment.")
elif best_score >= 0.70:
    print(f"\n ⚠️ ACCEPTABLE. Model is usable, but steering might be weak.")
else:
    print(f"\n ❌ POOR. Consider increasing 'candidate_layers_range' or checking data.")

print("\n" + "="*80 + "\n")

In [ ]:
import json
import os
import numpy as np

print("="*80)
print(" 🛠️ FIXING MISSING HISTORY FILE")
print("="*80 + "\n")

# Define the path
history_path = f"{config.local_save_dir}/training_history.json"

# Check if we have the data in memory
if 'histories' in globals() and best_layer in histories:
    print(f" Found training history for Layer {best_layer} in memory.")

    # Helper to convert numpy values (which JSON hates) to standard floats
    def clean_for_json(d):
        new_d = {}
        for k, v in d.items():
            if isinstance(v, list):
                new_d[k] = [float(x) for x in v]
            elif isinstance(v, (np.ndarray, np.generic)):
                new_d[k] = v.tolist()
            else:
                new_d[k] = v
        return new_d

    # Save it
    try:
        data_to_save = clean_for_json(histories[best_layer])
        with open(history_path, 'w') as f:
            json.dump(data_to_save, f, indent=2)
        print(f" ✅ Success! Created: {history_path}")
    except Exception as e:
        print(f" ❌ Error saving JSON: {e}")
else:
    # Fallback: Create a placeholder if memory was wiped
    print(" ⚠️ History variables not found in memory (did runtime restart?).")
    print("    Creating a placeholder file so upload succeeds.")
    dummy_data = {
        "status": "History data lost during session",
        "layer": best_layer,
        "final_accuracy": float(best_score)
    }
    with open(history_path, 'w') as f:
        json.dump(dummy_data, f, indent=2)
    print(f" ✅ Created placeholder: {history_path}")

print("\n👉 Now re-run the 'DEPLOYING TO HUGGINGFACE' cell.")

# Deploy Trained Model to HuggingFace with SafeTensors

In [ ]:
# Deploy Trained Model to HuggingFace (Safetensors Version)
import os
import shutil
import tempfile
import json
import torch
from huggingface_hub import HfApi, create_repo, login
from safetensors.torch import save_file  # Required for conversion
from datetime import datetime

print("="*80)
print(" 🚀 ROBUST DEPLOYMENT TO HUGGINGFACE (Safetensors)")
print("="*80 + "\n")

# --- 1. CONFIGURATION & AUTH ---
REPO_ID = config.hf_repo_name
TARGET_FOLDER_NAME = "bad_model_fairsteer_v2"

print("Step 1: Authentication")
hf_token = os.environ.get('HF_TOKEN', None)
if not hf_token:
    from getpass import getpass
    print("  HF_TOKEN not found in environment variables")
    hf_token = getpass("Enter HuggingFace Token: ")

try:
    login(token=hf_token)
    print("  ✅ Authenticated")
except Exception as e:
    print(f"  ❌ Failed: {e}")
    raise


print("\nStep 2: Locating Model Files")

possible_paths = [
    config.local_save_dir,
    f"./{TARGET_FOLDER_NAME}",
    f"/kaggle/working/{TARGET_FOLDER_NAME}",
    f"/content/{TARGET_FOLDER_NAME}",
    f"../{TARGET_FOLDER_NAME}"
]

SOURCE_DIR = None
# We look for pytorch_model.bin in source, but we will convert it later
required_files = ["pytorch_model.bin", "config.json", "scaler.pkl"]

for path in possible_paths:
    clean_path = os.path.abspath(path)
    if os.path.exists(clean_path):
        files_in_dir = os.listdir(clean_path)
        if all(f in files_in_dir for f in required_files):
            SOURCE_DIR = clean_path
            print(f"  ✅ FOUND VALID MODEL AT: {SOURCE_DIR}")
            break
        else:
            print(f"  ⚠️  Found folder at {clean_path}, but it is missing files.")

if not SOURCE_DIR:
    raise FileNotFoundError("Model files missing. Please check the Files sidebar.")

# --- 3. PREPARE UPLOAD ---
print("\nStep 3: Staging & Converting Files")
hf_upload_dir = tempfile.mkdtemp(prefix="bad_classifier_upload_")
print(f"  Temp Staging Area: {hf_upload_dir}")

# We define what to take from source, and how to handle it
files_to_process = [
    "pytorch_model.bin", # Will be converted
    "config.json",
    "scaler.pkl",
    "training_history.json",
    "training_dashboard.png"
]

upload_manifest = []

for filename in files_to_process:
    src = os.path.join(SOURCE_DIR, filename)

    if not os.path.exists(src):
        print(f"  ⚠️ Warning: {filename} not found (skipping)")
        continue

    # SPECIAL HANDLING: Convert PyTorch Bin to Safetensors
    if filename == "pytorch_model.bin":
        dst = os.path.join(hf_upload_dir, "model.safetensors")
        print(f"  🔄 Converting {filename} -> model.safetensors...")
        try:
            # Load weights from pickle
            state_dict = torch.load(src, map_location="cpu")
            # Save as safetensors
            save_file(state_dict, dst)
            size_mb = os.path.getsize(dst) / (1024 * 1024)
            print(f"  - Staged: model.safetensors ({size_mb:.2f} MB)")
            upload_manifest.append("model.safetensors")
        except Exception as e:
            print(f"  ❌ Conversion Failed: {e}")
            raise e
    else:
        # Standard Copy
        dst = os.path.join(hf_upload_dir, filename)
        shutil.copy(src, dst)
        size_mb = os.path.getsize(dst) / (1024 * 1024)
        print(f"  - Staged: {filename:<25} ({size_mb:.2f} MB)")
        upload_manifest.append(filename)

# Create README.md
readme_content = f"""---
license: apache-2.0
tags:
- fairsteer
- bias-detection
- tinyllama
- safetensors
library_name: safetensors
pipeline_tag: text-classification
---

# BAD Classifier for FairSteer

Biased Activation Detection (BAD) classifier for TinyLlama-1.1B.

## Artifacts
- **Model**: `model.safetensors` (SafeTensors format)
- **Scaler**: `scaler.pkl` (StandardScaler)
- **Config**: `config.json`

## Stats
- **Balanced Accuracy**: {best_score:.2%}
- **Best Layer**: {best_layer}
- **Training Date**: {datetime.now().strftime('%Y-%m-%d')}
"""
with open(f"{hf_upload_dir}/README.md", "w") as f:
    f.write(readme_content)
upload_manifest.append("README.md")

# --- 4. UPLOAD ---
print("\nStep 4: Uploading to HuggingFace")
api = HfApi()

try:
    create_repo(repo_id=REPO_ID, token=hf_token, private=config.hf_private, repo_type="model", exist_ok=True)

    print(f"  🚀 Uploading {len(upload_manifest)} files to {REPO_ID}...")

    api.upload_folder(
        folder_path=hf_upload_dir,
        repo_id=REPO_ID,
        repo_type="model",
        token=hf_token,
        commit_message=f"Upload trained model (Safetensors) Acc: {best_score:.2%}"
    )

    print("\n✅ DEPLOYMENT SUCCESSFUL!")
    print(f"🔗 URL: https://huggingface.co/{REPO_ID}")

except Exception as e:
    print(f"\n❌ Upload failed: {e}")
finally:
    shutil.rmtree(hf_upload_dir)
    print("  (Temp files cleaned up)")

# Quick Test After Deployment

In [ ]:
# Quick Test After Deployment
print("="*80)
print(" QUICK TEST: Loading from HuggingFace (Safetensors)")
print("="*80 + "\n")

from huggingface_hub import hf_hub_download
import pickle
import json
import torch
import os

try:
    repo_id = config.hf_repo_name
    print(f"Target Repo: {repo_id}")

    # 1. Download Model (Safetensors)
    print("Downloading model weights...")
    model_path = hf_hub_download(repo_id=repo_id, filename="model.safetensors")

    # 2. Download Config
    print("Downloading config...")
    config_path = hf_hub_download(repo_id=repo_id, filename="config.json")

    # 3. Download Scaler
    print("Downloading scaler...")
    scaler_path = hf_hub_download(repo_id=repo_id, filename="scaler.pkl")

    # 4. Validation checks
    # Check Config
    with open(config_path, 'r') as f:
        uploaded_config = json.load(f)

    # Check Scaler
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)

    print("\n✅ INTEGRITY CHECKS PASSED!")
    print("-" * 30)
    print(f"1. Model Config:")
    print(f"   - Layer Index: {uploaded_config.get('layer_idx', 'Unknown')}")
    print(f"   - Input Dim:   {uploaded_config.get('input_dim', 'Unknown')}")

    # Check if safetensors file actually exists locally
    if os.path.exists(model_path):
        print(f"2. Weights File:")
        print(f"   - Location: {model_path}")
        print(f"   - Size: {os.path.getsize(model_path) / 1024:.2f} KB")

    print(f"3. Scaler Properties:")
    print(f"   - Mean size:   {scaler.mean_.shape}")

    print(f"\n🎉 Model is publicly accessible and fully functional!")
    print("="*80 + "\n")

except Exception as e:
    print(f"\n❌ TEST FAILED: {e}")
    print("Possible reasons:")
    print(" 1. The upload hasn't finished processing yet.")
    print(" 2. You looked for 'pytorch_model.bin' but we uploaded 'model.safetensors'.")